# **ft-DECODER MODEL을 바탕으로 만들었습니다.**

In [2]:
pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 930.1 kB/s eta 0:00:00


In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from  sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler
from scipy import stats

import torch.nn.functional as F
from torch import nn, einsum

from einops import rearrange, repeat
from sklearn.preprocessing import LabelEncoder #라벨 인코더

Embedding 모듈

In [4]:
#연속형 변수 임베딩 함수
def embedfornum(df):
  linear=nn.Linear(4,20)
  L=[] #행단위로 리스트화를 진행했다
  for i in range(len(df.index)):
    x=df.iloc[i].tolist()
    z=pd.DataFrame(x)
    L.append(x)
  input=torch.Tensor(L)#리스트 텐서로 변환#이건 tensor로 전환후 하는 경우
  x=linear(input)
  return x

In [5]:
#임베딩텐서들 합치기
def AddEmbedding(x,y,z):
  proto_embedding=torch.cat((x,y), dim=1)
  final_embedding=torch.cat((proto_embedding,z), dim=1)
  final_embedding=final_embedding.tolist()
  final_embedding=np.array(final_embedding)
  fe=[]
  for i in range(len(x)):
    v=final_embedding[i].reshape(6,5).tolist()
    fe.append(v)
  final_embedding=torch.tensor(fe)
  return final_embedding

In [6]:
class Embedding(nn.Module):

  def __init__(self):

    super(Embedding, self).__init__()

    self.embedding=nn.Embedding(num_embeddings=3,embedding_dim=5)
    self.embedding1=nn.Embedding(num_embeddings=4,embedding_dim=5)
    self.label_encoders = {}
    self.label_encoders["type"] = LabelEncoder()
    self.label_encoders2 = {}
    self.label_encoders2["lane"] = LabelEncoder()
    #self.embedding2=embedfornum(df_num)


  def forward(self,x):
    x=x.fillna(0)
    df_num=x[["speed","capacity","cnt","od"]]
    df_cat=x[["lane","type"]]


    df_cat.loc[:, "type"] = self.label_encoders["type"].fit_transform(df_cat["type"])

    df_cat.loc[:, "lane"] = self.label_encoders2["lane"].fit_transform(df_cat["lane"])
    input=torch.LongTensor(df_cat["type"].values)
    embedding_type=self.embedding(input)
    input1=torch.LongTensor(df_cat["lane"].values)
    embedding_lane=self.embedding1(input1)
    embedding_num=embedfornum(df_num)
    embedding_final=AddEmbedding(embedding_num,embedding_type,embedding_lane)

    return embedding_final


트랜스포머(디코더), 어텐션 포함(기존ft-transformer 모델 참고)

In [7]:
#attention으로 나온 정보들을 합쳐주는 역할
def FeedForward(dim, mult = 4, dropout = 0.): #차원개수, 드롭아웃# mult는 내부게산을 위한 벡터 길이 설정값이다.
    return nn.Sequential(
        nn.LayerNorm(dim),
        nn.Linear(dim, dim * mult * 2),
        GEGLU(),
        nn.Dropout(dropout),
        nn.Linear(dim * mult, dim)
    )

In [8]:
class GEGLU(nn.Module):
    def forward(self, x):
        x, gates = x.chunk(2, dim = -1)#x를 2개로 분리
        return x * F.gelu(gates) # 그 다음 gate값은 활성화 함수 씌워주고 두 벡터끼리 다시 곱해준다.

In [9]:
#multihead attention(self attention)
class Attention(nn.Module):
    def __init__(
        self,
        dim,#차원개수 attention하고 싶은 차원개수
        heads = 8, #하이퍼 파라미터(본인이 정하는 것)(default 8임)
        dim_head = 64,#벡터들의 차원(default:64였음)
        dropout = 0 #드롭아웃비율(default=0)
    ):
        super().__init__()
        inner_dim = dim_head * heads #초기 벡터들의 차원(512)
        self.heads = heads
        self.scale = dim_head ** -0.5 #어텐션 스코어를 만드는것

        self.norm = nn.LayerNorm(dim)#정규화

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)#qkv벡터를 얻는 레이어(가중치 행렬) #forward에서 chunk로 따로 분리해주게 된다
        self.to_out = nn.Linear(inner_dim, dim, bias = False)#최종 아웃풋의 shape을 잡아주는 역할

        self.dropout = nn.Dropout(dropout)#드롭아웃(정확도를 높이는 효과)

    def forward(self, x):
        h = self.heads #

        x = self.norm(x) #벡터를 정규화

        q, k, v = self.to_qkv(x).chunk(3, dim = -1)#이 함수때문에 위에서 x3을 해준것, chunk로 분리해줌

        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), (q, k, v))#분리한 qkv에 lamda 함수를 맵핑하고 rearrange를 통해 head개수만큼 나눈다

        q = q * self.scale#스케일링#어텐션 스코어

        sim = einsum('b h i d, b h j d -> b h i j', q, k)# q벡터랑 k벡터 곱하는거#텐서연산 4차원텐서의 곱

        attn = sim.softmax(dim = -1)#소프트맥스 함수, attention distribution
        dropped_attn = self.dropout(attn)#정확도 높이고

        out = einsum('b h i j, b h j d -> b h i d', dropped_attn, v)# attention 분포 * values #텐서연산 4차원텐서의 곱
        out = rearrange(out, 'b h n d -> b n (h d)', h = h)#괄호의 의미는 h랑 d를 곱한다는 의미
        out = self.to_out(out)#attention의 결과물

        return out, attn

In [10]:
class Transformer(nn.Module):
    def __init__(  #모듈생성
        self,
        dim,#차원수
        depth,#layer개수
        heads,#헤드개수(병렬)
        dim_head,#q,k,v벡터길이(차원)를 정하는 파라미터
        attn_dropout,#드롭아웃비율
        ff_dropout#드롭아웃비율
    ):
        super().__init__() #nn.module method 다 불러오기
        self.layers = nn.ModuleList([]) # 모듈리스트 만들기

        for _ in range(depth):#layer개수만큼 attention과 feefforward를 실행해서 모듈리스트에 저장(디코더모듈 만들기)
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head, dropout = attn_dropout),
                FeedForward(dim, dropout = ff_dropout),
            ]))

    def forward(self, x, return_attn = False): #모듈값 계산
        post_softmax_attns = []

        for attn, ff in self.layers:#attention과 feedforward
            attn_out, post_softmax_attn = attn(x) #attention(x)의 return 값인 out과 attn값을 각각 할당
            post_softmax_attns.append(post_softmax_attn)#attn값은 리스트에 저장

            x = attn_out + x #return 값인 out을 계속 x에 더해 나간다.
            x = ff(x) + x #그리고 FeedForward의 return값도 더해 나감. x는 리스트형태의 input

        if not return_attn:#return_attn이 False이므로 x를 리턴한다.
            return x

        return x, torch.stack(post_softmax_attns)

# **모델**

In [11]:
class Model(nn.Module):
  def __init__(self, dim, depth, heads,dim_head,attn_dropout,ff_dropout,dimension_i,dimension_o):

    super(Model,self).__init__()

    self.embedding1= Embedding()
    self.transformer1=Transformer( dim, depth, heads,dim_head,attn_dropout,ff_dropout)
    self.predict1= nn.Sequential(nn.Flatten(), nn.Linear(dimension_i,dimension_o))

  def forward(self,x):
    x=self.embedding1(x)
    x=self.transformer1(x)
    x=self.predict1(x)

    return x

In [12]:
from torch.optim import AdamW

def train(model: Model,
          num_epochs: int,
          ground_truth: torch.tensor,
          input):
    loss_fn = nn.MSELoss()
    optimizer = AdamW(model.parameters(), lr = 0.001, weight_decay=0.1)

    for epoch in range(num_epochs):

        optimizer.zero_grad()

        prediction = model(input).reshape(len(input.index))
        print(prediction.shape)
        loss = loss_fn(prediction, ground_truth)
        loss.backward()
        optimizer.step()

        print(epoch+1, "epoch :", loss.item())

In [13]:
def df_divider(x_train,
               y_train,
               x_test,
               y_test,
               batch_size: int):
    train_length = int(len(x_train) / batch_size)
    test_length = int(len(x_test) / batch_size)

    train_dataset = [(x_train.iloc[idx*batch_size:(idx+1)*batch_size, :], torch.FloatTensor(y_train.iloc[idx*batch_size:(idx+1)*batch_size].values).reshape(-1, 1)) for idx in range(train_length)]
    train_dataset.append((x_train.iloc[train_length*batch_size:len(x_train), :], torch.FloatTensor(y_train.iloc[train_length*batch_size:len(x_train)].values).reshape(-1, 1)))
    test_dataset = [(x_test.iloc[idx*batch_size:(idx+1)*batch_size, :], torch.FloatTensor(y_test.iloc[idx*batch_size:(idx+1)*batch_size].values).reshape(-1, 1)) for idx in range(test_length)]
    test_dataset.append((x_test.iloc[test_length*batch_size:len(x_test), :], torch.FloatTensor(y_test.iloc[test_length*batch_size:len(x_test)].values).reshape(-1, 1)))

    return train_dataset, test_dataset

def new_train(model: Model,
              num_epochs: int,
              x_train,
              y_train,
              x_test,
              y_test,
              batch_size: int,
              opt_lr: float,#learning rate
              opt_wd: float,#weight decay
              save_model: bool):

    test_best_epoch = 0
    test_best_loss = 0.0

    loss_fn = nn.MSELoss()
    optimizer = AdamW(model.parameters(), lr = opt_lr, weight_decay = opt_wd)

    train_dataloader, test_dataloader = df_divider(x_train, y_train, x_test, y_test, batch_size)

    for epoch in range(num_epochs):
        running_loss = 0.0
        running_acc = 0.0

        model.train()

        for i, (x, ground_truth) in enumerate(train_dataloader, 0):
            optimizer.zero_grad()
            prediction = model(x)
            loss = loss_fn(prediction, ground_truth)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i%100 == 99:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

        model.eval()
        total_test_loss = 0.0
        div = 0

        with torch.no_grad():
            for data in test_dataloader:
                x, ground_truth = data
                prediction = model(x)
                total_test_loss += loss_fn(prediction, ground_truth)
                div += 1

        avg_loss = total_test_loss / div
        print('MSELoss for test dataset for epoch', epoch+1, ':', avg_loss.item())

        if epoch == 0:
            test_best_loss = avg_loss
        else:
            if test_best_loss > avg_loss:
                test_best_loss = avg_loss
                test_best_epoch = epoch
                if save_model:
                    path = './best_model.pth'
                    torch.save(model.state_dict(), path)

    print('The best model was the model from the epoch', test_best_epoch, 'which loss was', test_best_loss)
    if save_model:
        print('The best model is saved as best_model.pth')

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
data=pd.read_csv("")

In [16]:
data=data.fillna(0)

In [17]:
xx=data[["lane","type","speed","capacity","cnt","od"]]
yy=data["vol"]

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train,x_test,y_train,y_test=train_test_split(xx,yy,test_size=0.3)

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
model=Model(5,20,8,64,0.5,0.5,30,1)

In [22]:
new_train(model, 20, x_train, y_train, x_test, y_test, 16, 0.001, 0.1, False)

[1,   100] loss: 4067081.091
[1,   200] loss: 967972.263
[1,   300] loss: 961373.022
[1,   400] loss: 879909.167
MSELoss for test dataset for epoch 1 : 613906.3125
[2,   100] loss: 565947.590
[2,   200] loss: 484089.518
[2,   300] loss: 522170.768
[2,   400] loss: 444000.077
MSELoss for test dataset for epoch 2 : 351343.84375
[3,   100] loss: 395020.828
[3,   200] loss: 385886.597
[3,   300] loss: 400576.812
[3,   400] loss: 412199.708
MSELoss for test dataset for epoch 3 : 342633.6875
[4,   100] loss: 379431.896
[4,   200] loss: 362921.421
[4,   300] loss: 379909.710
[4,   400] loss: 397480.459
MSELoss for test dataset for epoch 4 : 332956.9375
[5,   100] loss: 362770.591
[5,   200] loss: 374720.795
[5,   300] loss: 392674.833
[5,   400] loss: 373706.649
MSELoss for test dataset for epoch 5 : 327933.5625
[6,   100] loss: 368816.633
[6,   200] loss: 374129.838
[6,   300] loss: 377237.211
[6,   400] loss: 378129.348
MSELoss for test dataset for epoch 6 : 332839.4375
[7,   100] loss: 355